In [2]:
import laspy
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree

# 📌 1. Load DEM from SHP file (2024)
def read_dem(shp_path):
    dem = gpd.read_file(shp_path)
    return dem

# 📌 2. Load point cloud (LAZ 2021)
def read_point_cloud(laz_path):
    las = laspy.read(laz_path)
    return np.vstack((las.x, las.y, las.z)).T  # XYZ matrix

# 📌 3. Compute height difference deltaH = Z_LAZ - Z_DEM
def compute_dh(point_cloud, dem):
    # Create a spatial tree with DEM points
    dem_coords = np.array(list(zip(dem.geometry.x, dem.geometry.y, dem['Z'])))
    dem_tree = cKDTree(dem_coords[:, :2])

    # Find the closest DEM point for each point in the cloud
    distances, indices = dem_tree.query(point_cloud[:, :2])

    # Get heights from DEM for the found points
    z_dem = dem_coords[indices, 2]

    # Calculate deltaH
    delta_h = point_cloud[:, 2] - z_dem
    return delta_h

# 📌 4. Compute accuracy metrics
def compute_accuracy(delta_h):
    metrics = {
        "Mean Error (Bias)": np.mean(delta_h),
        "Standard Deviation": np.std(delta_h),
        "RMSE": np.sqrt(np.mean(delta_h**2)),
        "Min ΔH": np.min(delta_h),
        "Max ΔH": np.max(delta_h),
    }
    return metrics

# 📌 Main function
def main():
    shp_path = "C:/Users/BARTEK/Desktop/exam/2/Lubin_2024_03_27_pc_t1.shp" 
    laz_path = "C:/Users/BARTEK/Desktop/exam/2/Lubin_2021_09_26_pc.laz" 

    dem = read_dem(shp_path)
    point_cloud = read_point_cloud(laz_path)
    delta_h = compute_dh(point_cloud, dem)
    metrics = compute_accuracy(delta_h)

    print("\n Results [cm]:")
    for key, value in metrics.items():
        print(f"{key}: {value:.4f}")

if __name__ == "__main__":
    main()



 Results [cm]:
Mean Error (Bias): 2.2757
Standard Deviation: 7.3809
RMSE: 7.7237
Min ΔH: -33.5400
Max ΔH: 47.5700
